# Optimization F24 Linear programming project
by Dmitry Beresnev (<d.beresnev@innopolis.university>)
and Vsevolod Klyushev (<v.klyushev@innopolis.university>)

Group ID = 2

In [20]:
from typing import Optional

import numpy as np

%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Utilities

### Decoding/encoding

In [21]:
def encoding_bin(mess: str, as_float:bool=False) -> tuple[np.ndarray, int]:
    """Encodes string into binary vector

    Args:
        mess (str): initial string
        as_float (bool): whether to convert output vector to floats. Defaults to False.

    Returns:
        tuple[np.ndarray, int]: (1d binary vector, number of bits per character)
    """

    # Convert each character to its ASCII value and then to binary
    xi = [format(ord(char), "08b") for char in mess]

    # Convert each binary string to a binary vector
    x = np.array([[int(bit) for bit in bit_str] for bit_str in xi])

    # Return the binary vector and its dimensions
    d = x.shape[1]  # Number of bits per character
    x = x.flatten() # convert into a 1-d vector

    if as_float:
        x = x.astype(np.float32)
    return x, d

def decoding_bin(x: np.ndarray, d:int) -> tuple[str, np.ndarray]:
    """Decodes a bunary vector into string

    Args:
        x (np.ndarray): 1d binary vector
        d (int): number of bits per character

    Returns:
        tuple[str, np.ndarray]: (decoded string, binary matrix)
    """

    # Ensure x is a binary vector (0s and 1s)
    x = np.clip(x, 0, 1)    # Clip values to be between 0 and 1
    x = np.round(x)         # Round values to the nearest integer

    # Initialize the output array
    y = np.zeros((len(x) // d, d), dtype=int)

    k = 0
    for i in range(len(x) // d):
        for j in range(d):
            y[i, j] = int(x[k])  # Fill the binary matrix
            k += 1

    # Convert binary to decimal and then to characters
    mess = "".join(chr(int("".join(map(str, row)), 2)) for row in y)

    return mess, y

In [22]:
# Try it
message_in = "So happy to see you"
print(f"Message sent: {message_in}")
binary_vector, dimensions = encoding_bin(message_in)
print(f"Dimensions: {dimensions}")
print(f"Binary Vector:\n{binary_vector}")
float_vector = binary_vector.astype(np.float32)
message_decoded, binary_matrix = decoding_bin(float_vector, dimensions)
print(f"Decoded message: {message_decoded}")

Message sent: So happy to see you
Dimensions: 8
Binary Vector:
[0 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0
 0 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1
 1 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0
 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1
 0 1 0 1]
Decoded message: So happy to see you


### Channel simulation

In [23]:
def noisy_channel(y: np.ndarray, percent_error:float, seed:Optional[int] = None)-> np.ndarray:
    """Simulates effect of the noisy channel.
    Disrupts percent_error% of y entries randomly

    Args:
        y (np.ndarray): input signal
        percent_error (float): percent of information to disrupt
        seed (Optional[int]): numpy random seed. Defaults to None.

    Returns:
        np.ndarray: noisy signal
    """

    if seed is not None:
        np.random.seed(seed)


    m = len(y)                                  # Length of the message
    k = int(np.floor(m * percent_error))        # Number of entries to corrupt
    indices = np.random.permutation(m)[:k]      # Random indices to corrupt
    y_n = np.copy(y)                            # Copy of the original message
    vec = np.random.rand(k) * np.mean(y)
    y_n[indices] = vec                          # Corruption of selected inputs
    return y_n

In [27]:
# Try it
message_in = "A crystal clear message"
float_vector, dimensions = encoding_bin(message_in, as_float=True)

y_prime = noisy_channel(float_vector, percent_error=0.05, seed=420)
print(f"Message sent: {message_in}")

message_corr_decoded, binary_matrix = decoding_bin(y_prime, dimensions)
print(f"Decoded noisy message: {message_corr_decoded}")

Message sent: A crystal clear message
Decoded noisy message: A cbyS0aL clear messaga


## Question 1

*Q1:  Model the problem as a linear program. Explain your reasoning*

Initial problem: $\min\limits_{x' \in \mathbb{R}^p} \| Ax'-y' \|_1$  such that $0 \leq x' \leq 1$

Note that $ \| Ax'-y' \|_1  = \sum_{i=1}^{m} |{(Ax')}_i-y'_i|$, where $( \cdot )_i$ - i-th component of vector.


Each element $|{(Ax')}_i-y'_i| = \max({(Ax')}_i-y'_i, y'_i -{(Ax')}_i)$ can be substituted with new variable $z'$ with the following additional
constraints: $z_i \geq {(Ax')}_i-y'_i$ and $z_i \geq y'_i-{(Ax')}_i$


Therefore, we can formulate the following linear programming problem, which is equivalent to the initial one:

$$
\begin{aligned}
& \min\limits_{x' \in \mathbb{R}^p, z \in \mathbb{R}^m} \sum_{i=1}^{m} z_i \\
\textbf{s.t. } &x' \geq 0 \\
&x' \leq 1 \\
& z_i \geq {(Ax')}_i-y'_i, i = 1 \dots m\\\
& z_i \geq y'_i-{(Ax')}_i, i = 1 \dots m \\
\end{aligned}
$$


## Question 2

*Q2:  Write this linear problem in standard form*

Let us firstly rewrite problem in **geometric** form:

$$
\begin{aligned}
& \min\limits_{z' \in \mathbb{R}^{p+m}} c^Tz' \\
\textbf{s.t. } &A' z' \geq b',
\end{aligned}
$$
where
$c \in \mathbb{R}^{p+m} = \sum_{i=p+1}^{p+m} e_i $,

$b' = (0_p, -1_p, -y', y')^T$,

$A' = \begin{pmatrix}
I_p & 0 \\
-I_p &  -1 \\
-A & I_m \\
A & I_m
\end{pmatrix} \in \mathbb{R}^{(2p+2m) \times (p+m)}
$,

$e_i$ - unit vector with 1 at index $i$ and all other zeroes,

$0_p$ - vector of $p$ zeroes,

$1_p$ - vector of $p$ ones,

$I_k$ - identity matrix of size $k \times k$.

Note, that by construction $z' = (x', z)^T \geq 0$, because $x' \geq 0$ by problem definition and $z \geq 0 $ by construction, as represents an absolute value.


Now we are ready to rewrite problem in **standard** form:

$$
\begin{aligned}
& \min\limits_{\tilde{x} \in \mathbb{R}^{2p+3m}} c^T\tilde{x} \\
\textbf{s.t. } &A' \tilde{x} = b \\
&\tilde{x} \geq 0,
\end{aligned}
$$
where
$c \in \mathbb{R}^{2p+3m} = \sum_{i=p+1}^{p+m} e_i $,

$b' = (-1_p, -y', y')^T$,

$A' = \begin{pmatrix}
-I_p &  -1 & -S^{1,p} \\
-A & I_m & -S^{p+1,p+m}\\
A & I_m & -S^{p+m+1,p+2m}
\end{pmatrix} \in \mathbb{R}^{(2p+2m) \times (p+m)}
$,

$S^{a,b}$ - slack variable matrix of size $(b-a+1) \times (p+2m)$ with rows $S^{a,b}_i = e_{a+i-1}$,

$1_p$ - vector of $p$ ones,

$I_k$ - identity matrix of size $k \times k$,

$e_i$ - unit vector with 1 at index $i$ and all other zeroes.